In [1]:
import pandas as pd
import re
import csv
import time
import numpy as np
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime
from user_class import User, find_new_friends
import scipy.sparse as sp

//anaconda/lib/python3.5/site-packages/IPython/core/magics/extension.py:47: UserWarning: %install_ext` is deprecated, please distribute your extension as a python package.
  "as a python package.", UserWarning)


Installed autotime.py. To use it, type:
  %load_ext autotime


In [2]:
input_dir = '../paymo_input/'
batch_file = 'batch_payment.csv'
stream_file = 'stream_payment.csv'
batch_path = input_dir + batch_file
stream_path = input_dir + stream_file

test_file = 'batch_payment.csv'
test_path = input_dir + test_file

time: 2.26 ms


In [10]:
## playing around with how to use regular expressions to handle the commas
# contained in the message section
batch_raw = []

with open(batch_path, newline='') as csvfile:
    for line in csvfile:
        batch_raw.append(line.strip())
        
df_batch = pd.DataFrame(batch_raw, columns = ['raw'])

## USE REGEX to parse out - everything before first comma is time, second comma is id1 and so on
## able to pick up messages with commas in them since it picks up all leftover characters at end of line
df_batch['time'] = df_batch['raw'].str.extract('^(.+?),', expand=True)
df_batch['id1'] = df_batch['raw'].str.extract('^[^,]*,([^,]*),', expand=True)
df_batch['id2'] = df_batch['raw'].str.extract('^[^,]*,[^,]*,([^,]*),', expand=True)
df_batch['amount'] = df_batch['raw'].str.extract('^[^,]*,[^,]*,[^,]*,([^,]*),', expand=True)
df_batch['message'] = df_batch['raw'].str.extract('^[^,]*,[^,]*,[^,]*,[^,]*,(.*$)', expand=True)

#can get rid of raw data now that we've parsed everything
df_batch = df_batch.drop('raw', 1)
df_batch = df_batch.drop(0, 0) #drop header row

time: 57.4 s


In [4]:
givers = df_batch.groupby('id1')
receivers = df_batch.groupby('id2')
partners_1 = {}
partners_2 = {}

time: 2.07 ms


In [11]:
df_batch.head()

,time,id1,id2,amount,message
0,time,id1,id2,amount,message
1,2016-11-02 09:38:53,49466,6989,23.74,🦄
2,2016-11-02 09:38:53,52349,8552,37.10,Pitcher
3,2016-11-02 09:38:53,32639,2562,18.68,🚕
4,2016-11-02 09:38:53,15381,13167,20.92,For your wife.


time: 8.95 ms


In [5]:
## find all transactions where user <user_id> was giver, then find list of partners in those transactions
for user_id,transactions in givers:
    
    #store list of all transaction partners as list (easiest type to extend later)
    try:
        partners_1[int(user_id)] = list(givers.get_group(user_id)['id2'].astype(int))
   
    #some lines of batch_payment.txt and stream_payment.txt are off - omit malformed entries
    except (KeyError, ValueError) as BadLine:
        print("Skipping invalid key:",user_id)
    
## same as before for all transactions where <user_id> was receiver
for user_id,transactions in receivers:
    
    #store list of all transaction partners as list (easiest type to extend later)
    try: 
        partners_2[int(user_id)] = list(receivers.get_group(user_id)['id1'].astype(int))
        
    #some lines of batch_payment.txt and stream_payment.txt are off - omit malformed entries
    except (KeyError, ValueError) as BadLine:
        print("Skipping invalid key:",user_id)

Skipping invalid key:  id1
Skipping invalid key:  no. Even if the union were a matter of economic indifference
Skipping invalid key:  and even if it were to be disadvantageous from the economic standpoint
Skipping invalid key:  id2
time: 1min 44s


In [11]:
## it's possible that some users only show up as givers and others only as receivers - combine to master list of all IDs
## in actuality for the provided batch_payment.txt all users show up as givers at least once, but not safe to assume
user_list_1 = np.array(list(partners_1.keys()))
user_list_2 = np.array(list(partners_2.keys()))
user_list = np.unique(np.concatenate([user_list_1,user_list_2]))

time: 23.5 ms


In [12]:
user_master_list = {}

#cycle through all users and agglomerate partners from all transactions
#conversion back and forth between list and numpy array is pretty fast
#lists easier to append to, hence why stored as list, but also wanted to use numpy.unique function.
for user_id in user_list:
    
    pp = []
    
    if user_id in partners_1.keys():
        pp += partners_1[user_id]
        
    if user_id in partners_2.keys():
        pp += partners_2[user_id]
        
    #reduce to (sorted) list of all unique partners
    user_master_list[user_id] = User(user_id, list(np.unique(pp)))

time: 2.32 s


In [60]:
## use the find_new_friends function (stored in user_class.py) to supplement friend tiers down to level of interest
#creating lists of friends down to 4th-degree connections takes about two minutes for 70,000 users on my macbook pro.
tier_depth = 4

#successively add tiers of friendship to every user in user_master_list
## this takes the longest of any part of the program - about an hour in total.
for tier in range(2,tier_depth+1):
    
    print("Building lists of connections of degree", tier, "for each user...")
    
    for user_id, user_data in user_master_list.items():
        user_data.friends[tier] = find_new_friends(user_master_list,user_data,tier)
            
print("Done. Connections of degree n accessible via User.friends[n]")

Building lists of connections of degree 2 for each user...
Building lists of connections of degree 3 for each user...


KeyboardInterrupt: 

time: 3h 6min 42s


In [69]:
user_master_list[2000].friends[3]

[5,
 7,
 9,
 10,
 13,
 14,
 16,
 20,
 21,
 28,
 29,
 30,
 37,
 39,
 41,
 45,
 53,
 69,
 73,
 77,
 78,
 82,
 87,
 90,
 96,
 100,
 109,
 129,
 138,
 142,
 144,
 161,
 169,
 178,
 181,
 188,
 195,
 196,
 197,
 199,
 204,
 210,
 211,
 212,
 213,
 214,
 215,
 218,
 219,
 221,
 223,
 224,
 226,
 227,
 229,
 231,
 232,
 233,
 234,
 235,
 236,
 237,
 238,
 240,
 241,
 242,
 245,
 247,
 248,
 249,
 250,
 252,
 254,
 255,
 256,
 257,
 258,
 259,
 260,
 261,
 264,
 266,
 268,
 270,
 271,
 273,
 274,
 275,
 276,
 277,
 278,
 279,
 280,
 285,
 286,
 287,
 288,
 289,
 291,
 293,
 295,
 296,
 297,
 298,
 299,
 300,
 302,
 304,
 305,
 306,
 307,
 310,
 311,
 312,
 314,
 315,
 316,
 317,
 319,
 322,
 323,
 325,
 328,
 329,
 330,
 333,
 336,
 339,
 341,
 342,
 345,
 346,
 347,
 348,
 352,
 353,
 354,
 355,
 357,
 358,
 363,
 365,
 366,
 369,
 370,
 372,
 373,
 378,
 379,
 380,
 382,
 383,
 384,
 385,
 386,
 387,
 390,
 391,
 393,
 2426,
 2428,
 2430,
 2432,
 2434,
 2435,
 2438,
 2439,
 2440,
 2442,
 244

time: 22.6 ms


In [70]:
## create forward lookup ##
## for each user, dictionary dos links a user id with the level of friendship
for user_id, user_data in user_master_list.items():
    user_data.build_dos()

time: 46.8 s


In [109]:
## the following tests take in pairs of user IDs as strings, convert them to int and use
## User.dos to look up whether they are connected to each other, and if so at what degree.

#first test - have these people had a transaction with each other in the batch data set?
def test1(id1,id2):
    
    if type(id1)==str and type(id2)==str:
    
        #converts user IDs from string into ints, which we use as dictionary keys
        try:
            user1 = int(id1)
        except ValueError:
            return 'unverified'

        try:
            user2 = int(id2)
        except ValueError:
            return 'unverified'
        
        #this being python, the following line doesn't take up new memory - just a shorthand
        if user1 in user_master_list.keys(): #have to check in case id1 is a new user
            user_dos = user_master_list[user1].dos

            if user2 in user_dos.keys():
                if user_dos[user2] == 1:
                    return 'trusted'
    
    return 'unverified'

#second test - is the transaction partner either a 1st or 2nd degree connection?    
def test2(id1,id2):
    
    if type(id1)==str and type(id2)==str:
    
        #converts user IDs from string into ints, which we use as dictionary keys
        try:
            user1 = int(id1)
        except ValueError:
            return 'unverified'

        try:
            user2 = int(id2)
        except ValueError:
            return 'unverified'
    
        #this being python, the following line doesn't take up new memory - just a shorthand
        if user1 in user_master_list.keys(): #have to check in case id1 is a new user

            user_dos = user_master_list[user1].dos

            if user2 in user_dos.keys():
                if user_dos[user2] in range (1,3):
                    return 'trusted'
    
    return 'unverified'
    
#third test - is the transaction partner at least a 4th degree connection?    
def test3(id1,id2):
    
    if type(id1)==str and type(id2)==str:
    
        #converts user IDs from string into ints, which we use as dictionary keys
        try:
            user1 = int(id1)
        except ValueError:
            return 'unverified'

        try:
            user2 = int(id2)
        except ValueError:
            return 'unverified'
        
        #this being python, the following line doesn't take up new memory - just a shorthand
        if user1 in user_master_list.keys(): #have to check in case id1 is a new user    

            user_dos = user_master_list[user1].dos

            if user2 in user_dos.keys():
                if user_dos[user2] in range(1,5):
                    return 'trusted'

    return 'unverified'

time: 116 ms


In [86]:
## Now load in second half of data set - this time we choose to flag transactions as verified or unverified.
stream_dict = {}
stream_dict['time'] = {}
stream_dict['id1'] = {}
stream_dict['id2'] = {}
stream_dict['amount'] = {}
stream_dict['message'] = {}

with open(stream_path, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for i, row in enumerate(reader):
        stream_dict['time'][i] = row['time']
        stream_dict['id1'][i] = row[' id1']
        stream_dict['id2'][i] = row[' id2']
        stream_dict['amount'][i] = row[' amount']
        stream_dict['message'][i] = row[' message']
        
df_stream = pd.DataFrame.from_dict(stream_dict)
df_stream = df_stream[['time','id1','id2','amount','message']]

time: 25.4 s


In [ ]:
## again, using regular expressions to parse from raw data.
stream_raw = []

with open(stream_path, newline='') as csvfile:
    for line in csvfile:
        stream_raw.append(line.strip())
        
df_stream = pd.DataFrame(stream_raw, columns = ['raw'])

## USE REGEX to parse out - everything before first comma is time, second comma is id1 and so on
## able to pick up messages with commas in them since it picks up all leftover characters at end of line
df_stream['time'] = df_stream['raw'].str.extract('^(.+?),', expand=True)
df_stream['id1'] = df_stream['raw'].str.extract('^[^,]*,([^,]*),', expand=True)
df_stream['id2'] = df_stream['raw'].str.extract('^[^,]*,[^,]*,([^,]*),', expand=True)
df_stream['amount'] = df_stream['raw'].str.extract('^[^,]*,[^,]*,[^,]*,([^,]*),', expand=True)
df_stream['message'] = df_stream['raw'].str.extract('^[^,]*,[^,]*,[^,]*,[^,]*,(.*$)', expand=True)

#can get rid of raw data now that we've parsed everything
df_stream = df_stream.drop('raw', 1)
df_stream = df_stream.drop(0, 0) #drop header row

In [75]:
df_stream.head()

,time,id1,id2,amount,message
0,2016-11-02 09:49:29,52575,1120,25.32,Spam
1,2016-11-02 09:49:29,47424,5995,19.45,Food for 🌽 😎
2,2016-11-02 09:49:29,76352,64866,14.99,Clothing
3,2016-11-02 09:49:29,20449,1552,13.48,LoveWins
4,2016-11-02 09:49:29,48676,19395,29.94,Jeffs still fat


time: 13 ms


In [76]:
df_stream['id2']

0            1120
1            5995
2           64866
3            1552
4           19395
5           45177
6           16725
7            8306
8           24692
9           66022
10           5637
11           5448
12          59830
13           9013
14           2942
15           3940
16           3197
17           5524
18          13233
19          49196
20          33106
21           2844
22          24068
23          26032
24           3827
25           3081
26          24675
27          21442
28          32471
29          47752
            ...  
2993519     48063
2993520      7015
2993521      5199
2993522     10096
2993523      9298
2993524     11303
2993525      3832
2993526     13925
2993527     33400
2993528     32430
2993529      3031
2993530        14
2993531     26160
2993532     22694
2993533      2908
2993534      6256
2993535     80358
2993536     17053
2993537     26631
2993538        59
2993539     73642
2993540     11765
2993541     38399
2993542      4282
2993543   

time: 126 ms


In [110]:
tst3 = df_stream[2000000:].apply(lambda x: test1(x['id1'], x['id2']), axis=1)
print(tst3)

2000000       trusted
2000001    unverified
2000002    unverified
2000003    unverified
2000004    unverified
2000005    unverified
2000006       trusted
2000007    unverified
2000008    unverified
2000009    unverified
2000010    unverified
2000011       trusted
2000012    unverified
2000013       trusted
2000014    unverified
2000015    unverified
2000016    unverified
2000017       trusted
2000018    unverified
2000019    unverified
2000020       trusted
2000021    unverified
2000022       trusted
2000023    unverified
2000024    unverified
2000025    unverified
2000026    unverified
2000027       trusted
2000028    unverified
2000029       trusted
              ...    
2993519    unverified
2993520       trusted
2993521       trusted
2993522    unverified
2993523    unverified
2993524       trusted
2993525    unverified
2993526    unverified
2993527    unverified
2993528    unverified
2993529    unverified
2993530       trusted
2993531    unverified
2993532    unverified
2993533   

In [118]:
## create output columns
## I found that pandas started to slow down drastically when trying to calculate over 1,000,000
## entries at a time...hence broken into pieces

#test 1 - immediate adjacency
test1_a = df_stream[:1000000].apply(lambda x: test1(x['id1'], x['id2']), axis=1)
test1_b = df_stream[1000000:2000000].apply(lambda x: test1(x['id1'], x['id2']), axis=1)
test1_c = df_stream[2000000:].apply(lambda x: test1(x['id1'], x['id2']), axis=1)
df_stream['test1'] = pd.concat([test1_a, test1_b, test1_c])

print(df_stream['test1'].value_counts())

KeyboardInterrupt: 

time: 8.39 s


In [121]:
print(df_stream['test1'].value_counts())

unverified    1539753
trusted       1453796
Name: test1, dtype: int64
time: 149 ms


In [122]:
#test 2
test2_a = df_stream[:1000000].apply(lambda x: test2(x['id1'], x['id2']), axis=1)
test2_b = df_stream[1000000:2000000].apply(lambda x: test2(x['id1'], x['id2']), axis=1)
test2_c = df_stream[2000000:].apply(lambda x: test2(x['id1'], x['id2']), axis=1)
df_stream['test2'] = pd.concat([test2_a, test2_b, test2_c])

print(df_stream['test2'].value_counts())

trusted       2342037
unverified     651512
Name: test2, dtype: int64
time: 1min 55s


In [114]:
#test 3
test3_a = df_stream[:1000000].apply(lambda x: test3(x['id1'], x['id2']), axis=1)
test3_b = df_stream[1000000:2000000].apply(lambda x: test3(x['id1'], x['id2']), axis=1)
test3_c = df_stream[2000000:].apply(lambda x: test3(x['id1'], x['id2']), axis=1)
df_stream['test3'] = pd.concat([test3_a, test3_b, test3_c])

print(df_stream['test3'].value_counts())

time: 1min 18s


In [116]:
## OUTPUT TO TEXT FILE ## 
input_dir = 'paymo_output/'
file_1 = input_dir + 'output1.txt'
file_2 = input_dir + 'output2.txt'
file_3 = input_dir + 'output3.txt'

df_stream['test1'].to_csv(file_1, index=False)
df_stream['test2'].to_csv(file_2, index=False)
df_stream['test3'].to_csv(file_3, index=False)


time: 6.64 s


In [117]:
df_stream['test1'].value_counts()

unverified    1539753
trusted       1453796
Name: test1, dtype: int64

time: 154 ms


In [10]:
## new approach - try to create a connectivity matrix - then, can figure out second-tier neighbors and such by multiplying matrix
# by itself
## conveniently, the list of users already ranges from 0 to 77,359 without gaps
## therefore, can use usernumbers directly as row/column indices in connectivity matrix

## takes about two minutes on 2016 macbook pro with 16GB of ram to convert to sparse matrix.

n_users = len(user_master_list.keys())
connectivity = np.zeros([n_users,n_users])

for user_id, user_data in user_master_list.items():
    for ff in user_data.friends[1]:
        #connections are bi-directional, so we fill in two spots in the connectivity matrix.
        connectivity[user_id,ff] = 1
        connectivity[ff,user_id] = 1
        
#multiplying 100k by 100k numpy matrices too taxing - try sparse matrices.
csp = sp.csr_matrix(connectivity)

time: 1min 50s


In [ ]:
#credit to Philipp Singer for his excellent tutorial on how to save the output of big matrix
#multiplication to disk: http://www.philippsinger.info/?p=464
l = n_users
 
f = tb.open_file('product_2.h5', 'w')
filters = tb.Filters(complevel=5, complib='blosc')
out = f.create_carray(f.root, 'data', tb.Float32Atom(), shape=(l, l), filters=filters)
 
bl = 1000 #this is the number of rows we calculate each loop
b = csp.tocsc() #we slice b on columns, csc improves performance
 
#slice by row
for i in range(0, l, bl):
    out[:,i:min(i+bl, l)] = (csp.dot(b[:,i:min(i+bl, l)])).toarray()
 
f.close()

In [17]:
h5 = tb.open_file('product_2.h5', 'r')
a = h5.root.data

## the advantage of this system is that only one row gets loaded into memory at a time -
# although we are saving a list of second-tier connections
for user_id in range(10):
    row = a[user_id,:] 
    user_master_list[user_id].friends[2] = list(np.nonzero(row))
    print(len(user_master_list[user_id].friends[2]))
    time.sleep(2)
    
h5.close()

(array([   0,    1,    2, ..., 4997, 4998, 4999]),)
(77360,)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
(array([   0,    1,    2, ..., 4993, 4997, 4999]),)
(77360,)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
(array([   0,    1,    2, ..., 4993, 4998, 4999]),)
(77360,)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
(array([   0,    1,    2, ..., 4996, 4997, 4999]),)
(77360,)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


KeyboardInterrupt: 

time: 7.11 s


In [27]:
h5.close()

time: 737 µs


In [6]:
#dictionary columns are randomly ordered - reorder as expected
df_batch = df_batch[['time','id1','id2','amount','message']]

time: 272 ms


In [6]:
## BIG PROBLEM - SKIPPED OVER A BUNCH OF ENTRIES
batch_dict['message'][377592]
#print(len(batch_dict['message'].keys()))

' 🇨🇴🇨🇴🇨🇴🇨🇴👍🏼🎉 '

In [5]:
batch_dict = {}
batch_dict['time'] = {}
batch_dict['id1'] = {}
batch_dict['id2'] = {}
batch_dict['amount'] = {}
batch_dict['message'] = {}

with open(test_path, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for i, row in enumerate(reader):
        batch_dict['time'][i] = row['time']
        batch_dict['id1'][i] = row[' id1']
        batch_dict['id2'][i] = row[' id2']
        batch_dict['amount'][i] = row[' amount']
        batch_dict['message'][i] = row[' message']
        
df_batch = pd.DataFrame.from_dict(batch_dict)

time: 28.2 s
